In [1]:
import random

class MasterMind():
    def __init__(self):
        self.code = str(random.randint(1,6)) + str(random.randint(1,6)) + str(random.randint(1,6)) + str(random.randint(1,6))
        self.attempts = random.randint(4,6)
        self.history = []
    
    def restart(self):
        self.code = str(random.randint(1,6)) + str(random.randint(1,6)) + str(random.randint(1,6)) + str(random.randint(1,6))
        self.attempts = random.randint(4,6)
        self.history = []
        print(self.code, self.attempts)
        return self.attempts    

    def get_code(self):
        return self.code    
    
    def get_attempts(self):
        return self.attempts
    
    def get_history(self):
        return self.history
    
    def evaluate_guess_code(self, guesscode):
        print(self.code, guesscode)
        code_status = [0,0,0,0]
        guesscode_status = [0,0,0,0]
        black_count = 0
        white_count = 0
        for i in range(4):
            if self.code[i] == guesscode[i]:
                black_count += 1
                code_status[i] = 1
                guesscode_status[i] = 1

        for i in range(4):
            if guesscode_status[i] == 0:
                for j in range(4):
                    if guesscode_status[i] == 0 and code_status[j] == 0 and guesscode[i] == self.code[j]:
                        guesscode_status[i] = 1
                        code_status[j] = 1
                        white_count += 1
        
        if black_count == 0 and white_count == 0:
            self.history.append([guesscode, "----"])
        else:
            self.history.append([guesscode, black_count*"B" + white_count*"W"])
        self.attempts -= 1
        
        print(black_count, white_count)
        if black_count == 0 and white_count == 0:
            return "----"
        else:
            return black_count*"B" + white_count*"W"

    # Task 2.3 (Cannot handle client v2 with no delay)
import socket
#socket.AF_INET => Using IPv4
#socket.SOCK_STREAM => Using TCP
listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#'127.0.0.1 loop back address, 12345 port number
listen_socket.bind(('127.0.0.1', 12345))
print("Server waiting connection...")
listen_socket.listen()
game_socket, addr = listen_socket.accept()
print("connection accepted:")

m = MasterMind()

done = False
while not done:
    # server waits for client's START, GUESS, STATUS or QUIT message
    rx_msg_str = ""
    while "\n" not in rx_msg_str:
        rx_msg_str += game_socket.recv(1024).decode()
    message = rx_msg_str[:-1]
    print("Received message: ", message)

    if message.startswith('START'):
        # option 1 start a game
        m.restart()
        tx_msg_str = "START," + str(m.get_attempts()) + "\n"
        game_socket.send(tx_msg_str.encode())
        print("Transmit:" + tx_msg_str[:-1])
    
    elif message.startswith('GUESS'):
        # option 2 to make a guess
        header, guesscode = message.split(",")
        #print(header, guesscode)
        result = m.evaluate_guess_code(guesscode)
        if result == "BBBB":
            tx_msg_str = "WON\n"
            done = True
        elif m.get_attempts() == 0:
            tx_msg_str = "LOSE,"+ m.get_code() +"\n"
            done = True
        else:
            tx_msg_str = "GUESS," + result + "\n"
        game_socket.send(tx_msg_str.encode())
        print("Transmit:" + tx_msg_str[:-1])
    
    elif message.startswith('HISTORY'):
        # option 3 to return attempts left and all past attempt results
        tx_msg_str = "HISTORY"
        tx_msg_str +=  "," + str(m.get_attempts())
        for guesscode, result in m.get_history():
            tx_msg_str += "," + guesscode + "," + result
        tx_msg_str += "\n"
        game_socket.send(tx_msg_str.encode())
        print("Transmit:" + tx_msg_str[:-1])

    elif message.startswith('QUIT'):
        done = True
        print("Server quits.")
    else:
        print("Received unknown message.")

game_socket.close()
listen_socket.close()

Server waiting connection...
connection accepted:
Received message:  START
5364 5
Transmit:START,5
Received message:  GUESS,1234
5364 1234
1 1
Transmit:GUESS,BW
Received message:  GUESS,5346
5364 5346
2 2
Transmit:GUESS,BBWW
Received message:  HISTORY
Transmit:HISTORY,3,1234,BW,5346,BBWW


KeyboardInterrupt: 